# Group Surveys Feature Determination

## Preparation

### Import

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)

## Function Definition

### Read and Concatenate Tables

In [ ]:
def read_collapse_tables(whats_features, meetings_features, surveys_features, grades):

  df_whats= pd.read_csv(whats_features, index_col='id',delimiter=";", header=0, usecols=['id', 'group', 'messages_sent', 'messages_total', 'contribution_index', 'ego_art', 'ego_nudges', 'alter_art', 'alter_nudges',
                                                                    'sentiment_avg', 'emotionality_avg', 'complexity_avg', 'influence_message_avg', 'influence_total_in',
                                                                    'influence_message_avg_in', 'influence_total', 'contribution_index_oscillation', 'activity_entanglement',
                                                                    'ALTERNATIVE_REALITIES_Treehugger', 'ALTERNATIVE_REALITIES_Fatherlander', 'ALTERNATIVE_REALITIES_Spiritualism',
                                                                    'ALTERNATIVE_REALITIES_Nerd',  'EMOTIONS_Fear', 'EMOTIONS_Happy',
                                                                    'EMOTIONS_Sad', 'EMOTIONS_Anger', 'Groupflow_Beeflow', 'Groupflow_Leechflow',
                                                                    'Groupflow_Antflow'])
  df_meetings= pd.read_excel(meetings_features,header=0,skiprows=range(1, 4)).rename_axis('Id')
  df_meetings.index += 1
  df_meetings= df_meetings.drop(columns=['group','shown_face','ID'], axis=1)
  df_surveys= pd.read_csv(surveys_features, index_col=0,usecols=['ID', 'group','ethical_likelihood', 'financial_likelihood', 'health_likelihood',
                                                                      'recreational_likelihood', 'social_likelihood', 'total_likelihood',
                                                                      'ethical_perceived', 'financial_perceived', 'health_perceived',
                                                                      'recreational_perceived', 'social_perceived', 'total_perceived',
                                                                      'O', 'C', 'E', 'A', 'N','harm_care_score', 'fairness_reciprocity_score',
                                                                      'in_group_loyality_score', 'authority_respect_score',
                                                                      'purity_sanctity_score', 'dummy_question1', 'dummy_question2',
                                                                      'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10', 'conservation', 'transcendence'])
  df_surveys= df_surveys.drop(columns=['group'], axis=1)
  df_grades= pd.read_excel(grades, index_col='ID', usecols=['ID','TOTAL TEORÍA (6)',	'COEVAL (1)',	'TRABAJO FINAL (1)'])
  df_grades.columns=['theory','coeval','project']
  df = pd.concat([df_meetings, df_whats, df_surveys, df_grades], axis=1)

  df = df.dropna()

  return df

### Apply Different Scalings

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

def process_scalings(df, scaler):
    if scaler == 'standard':
        scaler_obj = StandardScaler()
    elif scaler == 'minmax':
        scaler_obj = MinMaxScaler()
    elif scaler == 'robust':
        scaler_obj = RobustScaler()
    elif scaler == 'quartile':
        scaler_obj = QuantileTransformer(output_distribution='uniform', n_quantiles= len(df))
    else:
        raise ValueError("Invalid scaler. Please choose 'standard', 'minmax', 'robust', or 'quartile'.")

    scaled_df = df.copy()  # Create a copy of the original DataFrame

    # Scale the numerical columns
    numeric_cols = scaled_df.select_dtypes(include=['float64', 'int64']).columns
    scaled_df[numeric_cols] = scaler_obj.fit_transform(scaled_df[numeric_cols])

    return scaled_df


In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer
import pandas as pd
import numpy as np

def process_scalings(df, scaler):
    if scaler == 'standard':
        scaler_obj = StandardScaler()
    elif scaler == 'minmax':
        scaler_obj = MinMaxScaler()
    elif scaler == 'robust':
        scaler_obj = RobustScaler()
    elif scaler == 'quartile':
        scaler_obj = QuantileTransformer(output_distribution='uniform')
    else:
        raise ValueError("Invalid scaler. Please choose 'standard', 'minmax', 'robust', or 'quartile'.")

    scaled_df = df.copy()  # Create a copy of the original DataFrame

    # Filter and scale only the numerical columns
    numeric_cols = scaled_df.select_dtypes(include=['float64', 'int64']).columns
    scaled_df[numeric_cols] = scaler_obj.fit_transform(scaled_df[numeric_cols])

    return scaled_df

### Min-Max Scaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def min_max_scaling_df(df):
    scaler = MinMaxScaler()
    scaled_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    return scaled_df


### Display Correlation Matrix

In [ ]:
def corr_matrix(df):
  correlation_matrix = df.corr()

  # Create a figure and axes
  fig, ax = plt.subplots(figsize=(10, 8))

  # Create the heatmap using seaborn
  #sns.heatmap(correlation_matrix, cmap='coolwarm', ax=ax)

  # Set the title and axis labels
  ax.set_title('Correlation Matrix')
  ax.set_xlabel('Variables')
  ax.set_ylabel('Variables')

  # Rotate the x-axis labels for better readability
  plt.xticks(rotation=90)

  # Display the heatmap
  plt.show()

### Display Scatter Plot

In [ ]:
def scatterplot (df,x_column,y_column):
  import pandas as pd
  import seaborn as sns
  import matplotlib.pyplot as plt
  import numpy as np
  from sklearn.linear_model import LinearRegression
  from sklearn.preprocessing import PolynomialFeatures

  # Assuming your DataFrame is called 'df' and you want to check relationships between columns 'x' and 'y'
  x = df[x_column]
  y = df[y_column]

  # Create a scatter plot to visualize the relationship
  plt.scatter(x, y)
  plt.xlabel('x')
  plt.ylabel('y')
  plt.title('Scatter Plot')
  plt.show()

  # Fit a polynomial regression model
  degree = 2  # Degree of the polynomial
  poly_features = PolynomialFeatures(degree=degree)
  X_poly = poly_features.fit_transform(x.values.reshape(-1, 1))

  model = LinearRegression()
  model.fit(X_poly, y)

  # Calculate predicted values
  y_pred = model.predict(X_poly)

  # Plot the fitted curve
  plt.scatter(x, y)
  plt.plot(x, y_pred, color='red', label=f'Degree {degree} Polynomial Fit')
  plt.xlabel('x')
  plt.ylabel('y')
  plt.title('Polynomial Regression')
  plt.legend()
  plt.show()


### Correlation Significance

In [ ]:
def check_correlation_significance(df, col1, col2):
  import scipy.stats as stats

  # Assuming your DataFrame is called 'df'

  # Extract the 'performance' and 'recreational_perceived_mean' columns
  performance = df[col1]
  recreational_mean = df[col2]

  # Perform the hypothesis test
  corr_coeff, p_value = stats.pearsonr(performance, recreational_mean)

  # Print the results
  print("Correlation coefficient:", corr_coeff)
  print("p-value:", p_value)

  # Compare the p-value to the significance level
  alpha = 0.05  # Significance level
  if p_value < alpha:
      print("The correlation is statistically significant (reject H0).")
  else:
      print("The correlation is not statistically significant (fail to reject H0).")


### Apply Scalings given DF and print correlation

In [ ]:
def apply_scalings(df, column):
    correlations = []
    scalers = ['minmax', 'quartile']

    for scaler in scalers:
        df = process_scalings(df, scaler)

        # Calculate the correlations
        corr1 = df.corrwith(df[column], numeric_only=True)  # Set numeric_only parameter to True
        correlations.append(corr1)

    df_final = pd.concat(correlations, axis=1)
    df_final.columns = scalers

    return df_final

In [ ]:
def apply_scalings(df, column):

  correlations=[]
  scalers= ['quartile']

  for scaler in scalers:

    df= process_scalings(df, scaler)

    # Calculate the correlations
    corr1 = df.corrwith(df[column])
    #filtered_series = corr1[abs(corr1) > 0.6]
    correlations.append(corr1)

  df_final = pd.concat(correlations, axis=1)
  df_final.columns = scalers

  #fileterd_df = df_final[(np.abs(df) > 0.6).any(axis=1)]

  return df_final


## Use of Function

In [ ]:
grades = r'/content/drive/MyDrive/Projects/tps/grades/data/1_participants_grades.xlsx'
whats_features= r'/content/drive/MyDrive/Projects/tps/whatsapp/data/1_nodes_mixed.csv'
meetings_features= r'/content/drive/MyDrive/Projects/tps/meetings/data/12. features/4_individual_features_final_ratio.xlsx'
surveys_features=  r'/content/drive/MyDrive/Projects/tps/surveys/data/1_happimeter_individual_surveys.csv'

In [ ]:
df= read_collapse_tables(whats_features, meetings_features, surveys_features, grades)

In [ ]:
#df = min_max_scaling_df(df)
df.head(12)

,indiv_spoken_time,indiv_spoken_time_ratio,average_turn_duration,average_turn_duration_ratio,avg_time_without_speaking,avg_time_without_speaking_ratio,max_time_without_speaking,max_time_without_speaking_ratio,num_turns,num_turns_ratio,avg_turns_without_speaking,avg_turns_without_speaking_ratio,max_turns_without_speaking,max_turns_without_speaking_ratio,num_words,num_words_ratio,avg_words_turn,avg_words_turn_ratio,max_words_turn,max_words_turn_ratio,speech_neu,speech_ang,speech_hap,speech_sad,text_joy,text_anger,text_fear,text_sadness,group,messages_sent,messages_total,contribution_index,ego_art,ego_nudges,alter_art,alter_nudges,sentiment_avg,emotionality_avg,complexity_avg,influence_message_avg,influence_total_in,influence_message_avg_in,influence_total,contribution_index_oscillation,activity_entanglement,ALTERNATIVE_REALITIES_Treehugger,ALTERNATIVE_REALITIES_Fatherlander,ALTERNATIVE_REALITIES_Spiritualism,ALTERNATIVE_REALITIES_Nerd,EMOTIONS_Fear,EMOTIONS_Happy,EMOTIONS_Sad,EMOTIONS_Anger,Groupflow_Beeflow,Groupflow_Leechflow,Groupflow_Antflow,ethical_likelihood,financial_likelihood,health_likelihood,recreational_likelihood,social_likelihood,total_likelihood,ethical_perceived,financial_perceived,health_perceived,recreational_perceived,social_perceived,total_perceived,O,C,E,A,N,harm_care_score,fairness_reciprocity_score,in_group_loyality_score,authority_respect_score,purity_sanctity_score,dummy_question1,dummy_question2,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,conservation,transcendence,theory,coeval,project
6,1346.0,0.337682,8.518987,0.185779,19.887848,0.070957,169.82,0.044430,158.0,0.364055,1.740506,0.064235,9.0,0.028846,4025.0,0.329109,25.474684,0.187253,230.0,0.233503,0.768060,0.181860,0.043375,0.006704,0.297780,0.237582,0.227811,0.236827,2.0,22.0,69.0,-0.36,1.326.435.903,1.628.846.139,5.609.305.556,4.304.166.675,0.404612,0.268756,8.831.858.439,0.275771772,0.030908656,0.030908656,0.412769952,4.0,0.520000,0.362597,0.134296,0.001890,0.501216,0.152454,0.624326,0.145505,0.077715,0.332323,0.221886,0.445791,1.833333,3.833333,3.666667,6.166667,5.500000,4.200000,3.500000,4.000000,6.333333,4.333333,2.833333,4.0,0.683333,0.733333,0.883333,0.583333,0.416667,24.0,26.0,13.0,13.0,15.0,1.0,5.0,6.0,7.0,7.0,6.0,5.0,8.0,8.0,3.0,4.0,4.0,0.38,-1.59,4.64,0.78,8.5
8,298.0,0.074762,11.461538,0.249949,162.167200,0.578592,1529.54,0.400177,26.0,0.059908,15.384615,0.567781,125.0,0.400641,764.0,0.062469,29.384615,0.215993,167.0,0.169543,0.782264,0.203949,0.012210,0.001577,0.300151,0.233183,0.230322,0.236344,2.0,8.0,55.0,-0.71,3.302.708.333,3.037.500.024,1.476.083.333,2.912.499.994,0.746794,0.579783,988.272.047,0,0.959899291,0.959899291,0,4.0,0.529475,0.029491,0.125285,0.000506,0.844718,0.121923,0.685490,0.083875,0.108711,0.535406,0.037475,0.427119,4.666667,3.666667,4.333333,4.500000,4.666667,4.366667,3.666667,4.833333,6.000000,5.333333,4.333333,5.0,0.533333,0.683333,0.716667,0.500000,0.516667,28.0,20.0,21.0,23.0,24.0,3.0,3.0,4.0,6.0,5.0,4.0,6.0,4.0,5.0,4.0,6.0,8.0,1.77,-1.18,3.61,0.78,8.5
9,697.0,0.174862,8.822785,0.192404,47.935696,0.171028,1421.86,0.372004,79.0,0.182028,4.455696,0.164441,108.0,0.346154,2062.0,0.168602,26.101266,0.191859,257.0,0.260914,0.735439,0.228609,0.030550,0.005402,0.295444,0.239446,0.225465,0.239646,2.0,8.0,55.0,-0.71,2.095.583.333,3.866.666.635,1.623.819.444,2.291.666.687,0.428550,0.236425,9.147.384.644,0.061817313,0.254338987,0.127169494,0.061817313,1.0,0.518237,0.249223,0.057101,0.193481,0.500194,0.070225,0.553516,0.309070,0.067189,0.065323,0.311680,0.622997,3.000000,2.666667,3.166667,5.000000,5.000000,3.766667,3.666667,5.833333,4.333333,3.000000,2.833333,4.0,0.616667,0.700000,0.716667,0.716667,0.550000,17.0,23.0,16.0,13.0,14.0,2.0,4.0,6.0,7.0,6.0,4.0,4.0,3.0,6.0,2.0,6.0,4.0,1.72,-2.00,4.37,0.77,8.5
10,276.0,0.069242,6.272727,0.136793,25.912727,0.092453,218.84,0.057256,44.0,0.101382,3.113636,0.114911,24.0,0.076923,857.0,0.070074,19.477273,0.143169,78.0,0.079188,0.710042,0.214038,0.066733,0.009187,0.298211,0.236990,0.226584,0.238215,2

In [ ]:
df = df.drop(df[df['group'].isin([1, 3, 5])].index)
final_project= apply_scalings(df, 'coeval')
#final_project= final_project[(np.abs(final_project) > 0.6).any(axis=1)]
final_project.head(500)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:2627: UserWarning: n_quantiles (1000) is greater than the total number of samples (41). n_quantiles is set to n_samples.
  warnings.warn(
<ipython-input-44-2bb4c5b888f7>:11: FutureWarning: The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr1 = df.corrwith(df[column])


,quartile
indiv_spoken_time,0.120541
indiv_spoken_time_ratio,-0.099972
average_turn_duration,-0.135010
average_turn_duration_ratio,-0.030942
avg_time_without_speaking,-0.005142
avg_time_without_speaking_ratio,0.010546
max_time_without_speaking,0.192099
max_time_without_speaking_ratio,-0.010808
num_turns,0.151865
num_turns_ratio,0.119147


In [ ]:
df = df.drop([1, 3, 5])
max_theory= apply_scalings(df, 'theory')
max_theory= max_theory[(np.abs(max_theory) > 0.6).any(axis=1)]
max_theory.head(50)

In [ ]:
df_drop = df.drop([1, 3, 5])
max_theory= apply_scalings(df_drop, 'max_theory')
max_theory= max_theory[(np.abs(max_theory) > 0.6).any(axis=1)]
max_theory.head(50)

In [ ]:
apply_scalings(df, 'min_theory')